In [6]:
# -*- coding:utf-8  -*-
import pandas as pd
import time 
import sys
import jieba
from sklearn.decomposition import  LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pyLDAvis
import pyLDAvis.sklearn

In [7]:

def read_stopwords():
    stopwds = []
    with open ('chn-stop-words.txt' ,'r') as fh:
        for w in fh:
            stopwds.append(w.strip())
    fh.close()
    return stopwds
def covert_list_coding(mylist):
    result_list1 = [] # 先对list中的每一个元素encode()
    for i in mylist :
        if isinstance(i, list):  
            result_list1.append([j.encode("utf-8") for j in i ])
        else : 
            result_list1.append(i.encode("utf-8"))
#     print "len--> result_list1" , len(result_list1)

#    注意这里要重新开辟一个list,把上面的result_list1改成新的格式
    result_list2 = []
    for val in result_list1: # 遍历list1 把list1格式（list[[],[],[]]）转化为list2格式(list['sdfa' 'sdf' '2sd'])
        if val :
            result_list2.append(" ".join(val))
        else :
            result_list2.append(" ")
#     print "len--> result_list2" , len(result_list2)
    return result_list2

#    把df['content'] 转化为list[]
def cut_content(df, stopwds):
    cutted_content_list = []
    temp = None
    for val in df['title']:  # 这里 'content'  改为‘title’
        # 有可能为null
        temp = jieba.cut(str(val))
        temp = filter(lambda x : len(x) > 1 and x not in stopwds, temp)
        cutted_content_list.append(temp)
    return cutted_content_list

##   本函数没看懂 需进一步研究
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic #%d:" %topic_idx
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words -1 : -1]])
    print 

In [8]:
stopwds = read_stopwords()
df = pd.read_csv('all_data.csv',encoding='utf-8')
content_cutted = cut_content(df, stopwds)  # unicode  ## df  df2
std_content_cutted = covert_list_coding(content_cutted) # str
df['content_cutted'] = std_content_cutted
# df.head()

UnicodeEncodeError: 'ascii' codec can't encode character u'\u5728' in position 6: ordinal not in range(128)

In [5]:
    n_features = 1000
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,   # 0.5  10
                                min_df = 10)
    tf = tf_vectorizer.fit_transform(df.content_cutted)
    
    n_topics = 5
    lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50, 
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state= 0)
    lda.fit(tf)
    n_top_words =20
    tf_feature_names = tf_vectorizer.get_feature_names()
    print_top_words(lda, tf_feature_names, n_top_words)

AttributeError: 'DataFrame' object has no attribute 'content_cutted'

In [ ]:
pyLDAvis.enable_notebook()
#     pyLDAvis.show()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)